In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# from dynamicslib.common import f_
from dynamicslib.consts import muEM
from dynamicslib.continuation import f_df_CR3_single, find_bif, arclen_to_fail, arclen_cont
from dynamicslib.common import prop_ic, get_JC_tf, eom
from dynamicslib.integrator import dop853
from tqdm import tqdm
# from dynamicslib.targeter import *
from dynamicslib.plotting import plotly_display

file = "some/csv/path"
db = pd.read_csv(file).set_index("Index")

In [ ]:
def xtf2X(x0, tf):
    return np.array([x0[0], x0[2], x0[-2], tf / 2])


def X2xtf(X):
    return np.array([X[0], 0, X[1], 0, X[2], 0]), X[-1] * 2


def stmeom2DF(eomf, stm):
    dF = np.array(
        [
            [stm[1, 0], stm[1, 2], stm[1, -2], eomf[1]],
            [stm[-3, 0], stm[-3, 2], stm[-3, -2], eomf[-3]],
            [stm[-1, 0], stm[-1, 2], stm[-1, -2], eomf[-1]],
        ]
    )
    return dF


def f_func(x0, tf, xf):
    return np.array([xf[1], xf[-3], xf[-1]])


func = lambda X: f_df_CR3_single(X, X2xtf, stmeom2DF, f_func, False, muEM, 1e-11)

In [ ]:
X0 = db[["Initial x", "Initial z", "Initial vy", "Period"]].iloc[100].values
Xm1 = db[["Initial x", "Initial z", "Initial vy", "Period"]].iloc[99].values
X0[-1] /= 2
Xm1[-1] /= 2

In [ ]:
X0, dir0 = find_bif(
    Xm1,
    func,
    X0 - Xm1,
    1e-3,
    targ_tol=1e-11,
    bisect_tol=1e-7,
    bif_type="tangent",
    debug=False,
)

In [ ]:
# # switch direction:
# x0, tf = X2xtf(X0)
# ts, xs, _ = dop853(eom, (0, tf / 2), x0, 1e-12, 0)
# xs[1, -1] = 0
# X0 = xtf2X(xs[:, -1], ts[-1] * 2)

In [ ]:
X1 = X0.copy()
# X1[-1] *= 3 # if period multiplier
f, df, stm = func(X1)
svd = np.linalg.svd(df)
tangent2 = svd.Vh[-2]

In [ ]:
# p and m for plus and minus, if non-symmetric

In [ ]:
Xsp1, esp1 = arclen_cont(X1, func, tangent2, 1e-3, 0.1, 1e-11, use_exact_tangent=True)
Xsm1, esm1 = arclen_cont(X1, func, -tangent2, 1e-3, 0.1, 1e-10, use_exact_tangent=True)

In [ ]:
Xsp2, esp2 = arclen_cont(Xsp1[-1], func, Xsp1[-1] - Xsp1[-2], 5e-3, 1, 1e-10, max_iter=50)
Xsp3, esp3 = arclen_cont(Xsp2[-1], func, Xsp2[-1] - Xsp2[-2], 7e-3, 3, 1e-8, max_iter=50)

In [ ]:
# _, df, _ = func(Xsp2[-1])
# tangent = np.linalg.svd(df).Vh[-1]

# Xsp3, esp3 = arclen_to_fail(Xsp2[-1], func, tangent, 5e-3, 1e-8, 30)

In [ ]:
Xsm = [*Xsm1]
esm = [*esm1]
Xsp = [*Xsp1, *Xsp2, *Xsp3]
esp = [*esp1, *esp2, *esp3]

param_names = [
    "Index",
    "Initial x",
    "Initial z",
    "Initial vy",
    "Period",
    "Jacobi Constant",
    "Stability Index",
    "Eig1",
    "Eig2",
    "Eig3",
    "Eig4",
    "Eig5",
    "Eig6",
]

xyzs = []
data = []


iterator = list(enumerate(zip(Xsm, esm)))[::-1]
for i, (X, evals) in tqdm(iterator):
    Xcp = X.copy()
    xyzs.append(prop_ic(Xcp, X2xtf))
    args = np.argsort(np.abs(evals))
    stabind = np.max([(np.abs(lam) + 1 / np.abs(lam)) / 2 for lam in evals])
    jc, period = get_JC_tf(Xcp, X2xtf)
    data.append([-i, *Xcp[:-1], period, jc, stabind, *evals])

iterator = list(enumerate(zip(Xsp, esp)))
for i, (X, evals) in tqdm(iterator):
    Xcp = X.copy()
    xyzs.append(prop_ic(Xcp, X2xtf))
    args = np.argsort(np.abs(evals))
    stabind = np.max([(np.abs(lam) + 1 / np.abs(lam)) / 2 for lam in evals])
    jc, period = get_JC_tf(Xcp, X2xtf)
    data.append([i, *Xcp[:-1], period, jc, stabind, *evals])

In [ ]:
df = pd.DataFrame(data, columns=param_names).set_index("Index")
df.to_csv("save_name.csv")

In [ ]:
a = -800
b = None
c = 10
plotly_display(xyzs[a:b:c], df[a:b:c])